In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [23]:
from acnportal.acnsim.events import Event, EventQueue, PluginEvent, UnplugEvent, RecomputeEvent
from acnportal.acnsim.interface import Interface
from acnportal.acnsim.models import EV, Battery, EVSE
from acnportal.acnsim.network.sites import caltech_acn, jpl_acn
from acnportal.acnsim.simulator import Simulator
import gym
import numpy as np

from sustaingym.envs.evcharging.actions import get_action_space, to_schedule
from sustaingym.envs.evcharging.ev_charging import EVChargingEnv
from sustaingym.envs.evcharging.event_generation import AbstractTraceGenerator
from sustaingym.envs.evcharging.observations import get_observation_space, get_observation
from sustaingym.envs.evcharging.rewards import get_rewards
from sustaingym.envs.evcharging.utils import MINS_IN_DAY, DATE_FORMAT


In [7]:
acn = caltech_acn()
acn.station_ids[:7]

['CA-308', 'CA-508', 'CA-303', 'CA-513', 'CA-310', 'CA-506', 'CA-316']

In [186]:
class TestGenerator(AbstractTraceGenerator):
    def __init__(self,
                 site: str = "",
                 period: int = 5,
                 recompute_freq: int = 2,
                 date_range = None,
                 requested_energy_cap: float = 100):

        super().__init__(site, period, recompute_freq, date_range, requested_energy_cap)
    
    def update_day(self):
        return
    
    def get_event_queue(self):
        events = []
        for a, d, s in zip([0, 100, 100], [10, 150, 150], ['CA-508', 'CA-308', 'CA-303']):
            req_energy = 20.0
            battery = Battery(capacity=100, init_charge=max(0, 100-req_energy), max_power=100)
            ev = EV(arrival=a, departure=d, requested_energy=req_energy, station_id=s, session_id=str(a), battery=battery, estimated_departure=a+50)
            
            events.append(PluginEvent(a, ev))
        # events.append(RecomputeEvent(110))
        eq = EventQueue(events)
        return eq, 3

In [187]:
tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
eq, _ = tg.get_event_queue()

In [188]:
eq.queue

[(0,
  acnportal.acnsim.events.event.PluginEvent(timestamp=0, event_type=<str object at 0x7f9d5c7a0e70>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f9d3c036d60>)),
 (100,
  acnportal.acnsim.events.event.PluginEvent(timestamp=100, event_type=<str object at 0x7f9d5c7a0e70>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f9d3c036100>)),
 (100,
  acnportal.acnsim.events.event.PluginEvent(timestamp=100, event_type=<str object at 0x7f9d5c7a0e70>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f9d3c036fd0>))]

In [189]:
eq.get_current_events(45)

[acnportal.acnsim.events.event.PluginEvent(timestamp=0, event_type=<str object at 0x7f9d5c7a0e70>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f9d3c036d60>)]

In [190]:
from collections import defaultdict

# send 16 A in

tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
env = EVChargingEnv(tg, action_type='discrete')
for _ in range(1):
    observation = env.reset()

    offline_reward_calc = 0.
    for event in env.events.queue:
        if isinstance(event[1], PluginEvent):
            offline_reward_calc += (event[1].ev.departure - event[1].ev.arrival) * env.period * 16

    rewards = 0.
    done = False
    i = 0
    action = np.ones(shape=(54,), ) * 2
    d = defaultdict(list)
    while not done:
        observation, reward, done, info = env.step(action)
        rewards += reward
        i += 1

    print()
    print()
    print("total iterations: ", i)
    print("environment reward: ", rewards)
    print("offline reward calc: ", offline_reward_calc)



total iterations:  4
environment reward:  -1244160.0
offline reward calc:  8800.0


In [185]:
# 0  50 11920.0  12000
# 0 100 13769.23 16000
# 0 150 13769.23 20000
# 0 200 13769.23 24000

11200

11200